- 系统信息：包括CPU、GPU、内存、硬盘等

# 基本设置

In [1]:
import psutil
import pynvml
import time
import datetime
import os

def getProcess(pName):
    # 获取当前系统所有进程id列表
    all_pids  = psutil.pids()

    # 遍历所有进程，名称匹配的加入process_lst
    process_lst = []
    for pid in all_pids:
        p = psutil.Process(pid)
        if (p.name() == pName):
            process_lst.append(p)

    return process_lst

# CPU/GPU/Memory

## 总体

In [2]:
psutil.test() 

USER         PID %MEM     VSZ     RSS TTY           START    TIME  COMMAND
root           1    ?  225712    9660 ?             Dec17   59:09  systemd
root           2    ?       ?       ? ?             Dec17   00:00  kthreadd
root           4    ?       ?       ? ?             Dec17   00:00  kworker/0:0H
root           5    ?       ?       ? ?             Dec17   00:00  kworker/u64:0
root           7    ?       ?       ? ?             Dec17   00:00  mm_percpu_wq
root           8    ?       ?       ? ?             Dec17   00:00  ksoftirqd/0
root           9    ?       ?       ? ?             Dec17   01:14  rcu_sched
root          10    ?       ?       ? ?             Dec17   00:00  rcu_bh
root          11    ?       ?       ? ?             Dec17   00:00  migration/0
root          12    ?       ?       ? ?             Dec17   00:02  watchdog/0
root          13    ?       ?       ? ?             Dec17   00:00  cpuhp/0
root          14    ?       ?       ? ?             Dec17   00:00  cpuh

gdm         1460  0.1  354432   22800 /dev/tty1     Dec17   00:00  gsd-clipboard
gdm         1464  0.1  606952   28796 /dev/tty1     Dec17   18:17  gsd-color
gdm         1466    ?  393704   13116 /dev/tty1     Dec17   00:00  gsd-datetime
gdm         1467    ?  283736    4744 /dev/tty1     Dec17   00:00  gsd-housekeeping
gdm         1469  0.1  506664   22820 /dev/tty1     Dec17   00:00  gsd-keyboard
gdm         1471  0.1 1088344   25900 /dev/tty1     Dec17   00:00  gsd-media-keys
gdm         1473    ?  201992    4140 /dev/tty1     Dec17   00:00  gsd-mouse
gdm         1476  0.1  525960   24612 /dev/tty1     Dec17   00:02  gsd-power
gdm         1479    ?  267004    8064 /dev/tty1     Dec17   00:00  gsd-print-notifications
gdm         1481    ?  202012    4068 /dev/tty1     Dec17   00:00  gsd-rfkill
gdm         1482    ?  275728    4608 /dev/tty1     Dec17   00:00  gsd-screensaver-proxy
gdm         1483    ?  321428    9876 /dev/tty1     Dec17   00:00  gsd-sharing
gdm         1487    ?  39

### 用户

In [3]:
print('开机时间: ', datetime.datetime.fromtimestamp(psutil.boot_time ()).strftime("%Y-%m-%d %H: %M: %S"))
users_count = len(psutil.users())
print('用户数: ', users_count)
for u in psutil.users():print(u)

开机时间:  2018-12-17 15: 19: 11
用户数:  9
suser(name='root', terminal='pts/0', host='192.168.10.222', started=1546563840.0, pid=13708)
suser(name='root', terminal='pts/1', host='192.168.10.222', started=1546563968.0, pid=13782)
suser(name='root', terminal='pts/2', host='192.168.10.222', started=1546564096.0, pid=14084)
suser(name='root', terminal='pts/3', host='192.168.10.222', started=1546564480.0, pid=14315)
suser(name='root', terminal='pts/4', host='192.168.10.222', started=1546564480.0, pid=14397)
suser(name='root', terminal='pts/5', host='192.168.0.104', started=1546565120.0, pid=14846)
suser(name='root', terminal='pts/6', host='192.168.0.104', started=1546565504.0, pid=15007)
suser(name='root', terminal='pts/7', host='192.168.10.135', started=1546565760.0, pid=15150)
suser(name='wiseweb', terminal=':1', host=':1', started=1545716224.0, pid=30054)


### GPU

In [4]:
pynvml.nvmlInit()
print('显示驱动信息: ')
print("Driver: ", pynvml.nvmlSystemGetDriverVersion()) 
print('--------------')
print('设备信息: ')
deviceCount = pynvml.nvmlDeviceGetCount()
print('  共 %s 块 GPU，名称为：'%deviceCount)
for i in range(deviceCount):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    print("    GPU", i, ":", pynvml.nvmlDeviceGetName(handle))
print('--------------')
for i in range(deviceCount):
    print('查看第 %s 块GPU的显存、温度、风扇、电源: '%i)
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print("Memory Total: %0.2f G"%(info.total/1024/1024/1024)) # 总的显存大小
    print("Memory Free: %0.2f G "%(info.free/1024/1024/1024)) # 剩余显存大小
    print("Memory Used: %0.2f G "%(info.used/1024/1024/1024)) 
    print("Memory Used percent: %0.2f %% "%(info.used/info.total*100))
    print("Temperature is %d C"%(pynvml.nvmlDeviceGetTemperature(handle,0)))
    try :
        print("Fan speed is ",pynvml.nvmlDeviceGetFanSpeed(handle))
    except :
        continue
    print("Power ststus",pynvml.nvmlDeviceGetPowerState(handle))
    print('--------------')

#最后要关闭管理工具
pynvml.nvmlShutdown()

显示驱动信息: 
Driver:  b'390.67'
--------------
设备信息: 
  共 2 块 GPU，名称为：
    GPU 0 : b'GeForce GTX 1080 Ti'
    GPU 1 : b'GeForce GTX 1080 Ti'
--------------
查看第 0 块GPU的显存、温度、风扇、电源: 
Memory Total: 10.92 G
Memory Free: 10.79 G 
Memory Used: 0.12 G 
Memory Used percent: 1.12 % 
Temperature is 36 C
Fan speed is  0
Power ststus 8
--------------
查看第 1 块GPU的显存、温度、风扇、电源: 
Memory Total: 10.92 G
Memory Free: 10.91 G 
Memory Used: 0.00 G 
Memory Used percent: 0.02 % 
Temperature is 27 C
Fan speed is  0
Power ststus 8
--------------


### CPU 

In [5]:
print('CPU物理个数: ', psutil.cpu_count(logical=False))
print('CPU逻辑个数: ', psutil.cpu_count())
print()
print('系统的CPU利用率: ')
print(psutil.cpu_percent(interval = 2))
print()
print('每块的CPU利用率: ')
print(psutil.cpu_percent(interval = 2, percpu = True))

CPU物理个数:  16
CPU逻辑个数:  32

系统的CPU利用率: 
0.2

每块的CPU利用率: 
[0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.0]


In [4]:
print(psutil.cpu_percent(interval = 2, percpu = True))

[1.0, 1.0, 0.0, 0.5, 0.5, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


### Memory

In [16]:
memory = psutil.virtual_memory()
# print('内存利用率: ', psutil.virtual_memory().percent)
print("Memory Total: %0.2f G"%(memory.total/1024/1024/1024)) # 总的内存大小
print("Memory Free: %0.2f G "%(memory.free/1024/1024/1024)) # 剩余内存大小
memory_used = memory.total - memory.free
print("Memory Used: %0.2f G "%(memory_used/1024/1024/1024)) 
print("Memory Used percent: %0.2f %% "%(memory_used/memory.total * 100))
print('--------------')
print("Memory Used percent: %0.2f %% "%(memory.percent))

Memory Total: 31.36 G
Memory Free: 5.08 G 
Memory Used: 26.29 G 
Memory Used percent: 83.82 % 
--------------
Memory Used percent: 39.30 % 


### 硬盘

In [7]:
# print('磁盘使用情况: ', psutil.disk_usage('/'))
# disk = psutil.disk_usage('C:\\')
disk = psutil.disk_usage('/') # 文件所在硬盘
# print('硬盘利用率: ', disk.percent)
print("Disk Total: %0.2f G"%(disk.total/1024/1024/1024)) # 总的内存大小
print("Disk Free: %0.2f G "%(disk.free/1024/1024/1024)) # 剩余内存大小
print("Disk Used: %0.2f G "%(disk.used/1024/1024/1024)) 
print("Disk Used percent: %0.2f %% "%(disk.used/disk.total  * 100))
print('--------------')
print("Disk Used percent: %0.2f %% "%(disk.percent))

psutil.disk_partitions()  # 磁盘分配

Disk Total: 228.23 G
Disk Free: 178.20 G 
Disk Used: 38.37 G 
Disk Used percent: 16.81 % 
--------------
Disk Used percent: 17.70 % 


[sdiskpart(device='/dev/sda1', mountpoint='/', fstype='ext4', opts='rw,relatime,errors=remount-ro,data=ordered'),
 sdiskpart(device='/dev/loop0', mountpoint='/snap/gnome-characters/103', fstype='squashfs', opts='ro,nodev,relatime'),
 sdiskpart(device='/dev/loop1', mountpoint='/snap/gtk-common-themes/319', fstype='squashfs', opts='ro,nodev,relatime'),
 sdiskpart(device='/dev/loop2', mountpoint='/snap/gnome-3-26-1604/70', fstype='squashfs', opts='ro,nodev,relatime'),
 sdiskpart(device='/dev/loop3', mountpoint='/snap/core/4917', fstype='squashfs', opts='ro,nodev,relatime'),
 sdiskpart(device='/dev/loop5', mountpoint='/snap/gnome-calculator/180', fstype='squashfs', opts='ro,nodev,relatime'),
 sdiskpart(device='/dev/loop4', mountpoint='/snap/gnome-system-monitor/51', fstype='squashfs', opts='ro,nodev,relatime'),
 sdiskpart(device='/dev/loop6', mountpoint='/snap/gnome-logs/37', fstype='squashfs', opts='ro,nodev,relatime'),
 sdiskpart(device='/dev/loop7', mountpoint='/snap/core/6034', fstype=

In [8]:
# read_count：读取次数
# write_count：写入次数
# read_bytes：读取的字节数
# write_bytes：写入的字节数
# read_time(磁盘读时间)
# write_time(磁盘写时间)
print('磁盘读写计数器: ', psutil.disk_io_counters())
# perdisk=True参数获取单个分区IO个数    
psutil.disk_io_counters(perdisk=True) 

磁盘读写计数器:  sdiskio(read_count=173615, write_count=631045, read_bytes=2267797504, write_bytes=310978326528, read_time=131792, write_time=11119024, read_merged_count=2064, write_merged_count=881443, busy_time=356268)


{'loop0': sdiskio(read_count=54, write_count=0, read_bytes=360448, write_bytes=0, read_time=4, write_time=0, read_merged_count=0, write_merged_count=0, busy_time=0),
 'loop1': sdiskio(read_count=486, write_count=0, read_bytes=1143808, write_bytes=0, read_time=4, write_time=0, read_merged_count=0, write_merged_count=0, busy_time=0),
 'loop2': sdiskio(read_count=134, write_count=0, read_bytes=1169408, write_bytes=0, read_time=0, write_time=0, read_merged_count=0, write_merged_count=0, busy_time=0),
 'loop3': sdiskio(read_count=1459, write_count=0, read_bytes=2531328, write_bytes=0, read_time=768, write_time=0, read_merged_count=0, write_merged_count=0, busy_time=24),
 'loop4': sdiskio(read_count=40, write_count=0, read_bytes=117760, write_bytes=0, read_time=0, write_time=0, read_merged_count=0, write_merged_count=0, busy_time=0),
 'loop5': sdiskio(read_count=45, write_count=0, read_bytes=122880, write_bytes=0, read_time=0, write_time=0, read_merged_count=0, write_merged_count=0, busy_tim

### 网络

In [9]:
print('网络接口状态:')
for index, key in enumerate(psutil.net_if_stats()):
    print('  --  %s  %s'%(index +1, key))
    print(psutil.net_if_stats()[key])
print()
print('网卡信息: ')
for index, key in enumerate(psutil.net_if_addrs()):
    print('  --  %s  %s'%(index +1, key))
    print(psutil.net_if_addrs()[key])

网络接口状态:
  --  1  enp7s0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=1000, mtu=1500)
  --  2  enp6s0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=65535, mtu=1500)
  --  3  lo
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=65536)

网卡信息: 
  --  1  lo
[snicaddr(family=<AddressFamily.AF_INET: 2>, address='127.0.0.1', netmask='255.0.0.0', broadcast=None, ptp=None), snicaddr(family=<AddressFamily.AF_INET6: 10>, address='::1', netmask='ffff:ffff:ffff:ffff:ffff:ffff:ffff:ffff', broadcast=None, ptp=None), snicaddr(family=<AddressFamily.AF_PACKET: 17>, address='00:00:00:00:00:00', netmask=None, broadcast=None, ptp=None)]
  --  2  enp7s0
[snicaddr(family=<AddressFamily.AF_INET: 2>, address='192.168.30.220', netmask='255.255.255.0', broadcast='192.168.30.255', ptp=None), snicaddr(family=<AddressFamily.AF_INET6: 10>, address='fe80::280b:e6bf:7518:d97a%enp7s0', netmask='ffff:ffff:ffff:ffff::', broadcast=None, ptp=None), snicad

In [10]:
# bytes_sent:发送的字节数
# bytes_recv:收到的字节数
# packets_sent:发送的数据包数量
# packets_recv:收到的数据包shul
# errin：接收时的错误数
# errout:发送时的错误数
# dropin:丢弃的传入数据包总数
# dropout：丢弃的传出数据包总数
print('系统网络总的IO信息: ', psutil.net_io_counters())
# pernic=True时输出每个网络接口的信息
psutil.net_io_counters(pernic = True)

系统网络总的IO信息:  snetio(bytes_sent=1822996346, bytes_recv=20078338143, packets_sent=10631578, packets_recv=16631426, errin=17, errout=0, dropin=16, dropout=0)


{'enp7s0': snetio(bytes_sent=1703769690, bytes_recv=19959111487, packets_sent=10155106, packets_recv=16154954, errin=17, errout=0, dropin=16, dropout=0),
 'enp6s0': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'lo': snetio(bytes_sent=119229494, bytes_recv=119229494, packets_sent=476477, packets_recv=476477, errin=0, errout=0, dropin=0, dropout=0)}

In [15]:
print('当前网络连接信息:')
for index, key in enumerate(psutil.net_connections(kind = 'inet')):
    if key.laddr.port in [8888, 8097, 6006, 9001]:  # 本地
        print('  - 本地  %s  ip :%s  port :%s  status :%s  pid :%s'%(index +1, 
                                                                key.laddr.ip, 
                                                                key.laddr.port, 
                                                                key.status, 
                                                                key.pid))
#         print(key)
    if len(key.raddr) > 0: 
        if key.raddr.port in [8888, 8097, 6006, 9001]:  # 远程
            print('  * 远程  %s  laddr ip :%s  laddr port :%s  raddr ip :%s  raddr port :%s  status :%s  pid :%s'%(index +1,
                                                                                                               key.laddr.ip,
                                                                                                               key.laddr.port,
                                                                                                               key.raddr.ip,
                                                                                                               key.raddr.port, 
                                                                                                               key.status,
                                                                                                               key.pid))
#             print(key)

当前网络连接信息:
  - 本地  2  ip :0.0.0.0  port :8097  status :LISTEN  pid :1646
  - 本地  9  ip :192.168.30.220  port :9001  status :LISTEN  pid :11191
  - 本地  56  ip :192.168.30.220  port :9001  status :ESTABLISHED  pid :11191
  - 本地  62  ip :192.168.30.220  port :9001  status :ESTABLISHED  pid :11191
  - 本地  72  ip :0.0.0.0  port :6006  status :LISTEN  pid :10597
  - 本地  73  ip :::  port :8097  status :LISTEN  pid :1646


## 特定

In [12]:
all_pids  = psutil.pids() # 所有进程ID
# process_lst = []
# for pid in all_pids:
#     p = psutil.Process(pid)
#     process_lst.append(p)

In [13]:
p = psutil.Process(os.getpid())
print(p.create_time)
# datetime.datetime.fromtimestamp(p.create_time).strftime("%Y-%m-%d %H:%M:%S")

<bound method Process.create_time of psutil.Process(pid=15257, name='python3', started='09:40:28')>


In [14]:
# 查看系统全部进程
for pnum in psutil.pids():
    p = psutil.Process(pnum)
    print(u"进程名 %-20s  内存利用率 %-18s 进程状态 %-10s 创建时间 %-10s " \
    % (p.name(), p.memory_percent(), p.status(), p.create_time()))

进程名 systemd               内存利用率 0.02937644928378027 进程状态 sleeping   创建时间 1545031151.04 
进程名 kthreadd              内存利用率 0.0                进程状态 sleeping   创建时间 1545031151.04 
进程名 kworker/0:0H          内存利用率 0.0                进程状态 idle       创建时间 1545031151.04 
进程名 kworker/u64:0         内存利用率 0.0                进程状态 idle       创建时间 1545031151.04 
进程名 mm_percpu_wq          内存利用率 0.0                进程状态 idle       创建时间 1545031151.04 
进程名 ksoftirqd/0           内存利用率 0.0                进程状态 sleeping   创建时间 1545031151.04 
进程名 rcu_sched             内存利用率 0.0                进程状态 idle       创建时间 1545031151.04 
进程名 rcu_bh                内存利用率 0.0                进程状态 idle       创建时间 1545031151.04 
进程名 migration/0           内存利用率 0.0                进程状态 sleeping   创建时间 1545031151.04 
进程名 watchdog/0            内存利用率 0.0                进程状态 sleeping   创建时间 1545031151.04 
进程名 cpuhp/0               内存利用率 0.0                进程状态 sleeping   创建时间 1545031151.04 
进程名 cpuhp/1               内存利用率 0.0       

进程名 ksmd                  内存利用率 0.0                进程状态 sleeping   创建时间 1545031151.25 
进程名 khugepaged            内存利用率 0.0                进程状态 sleeping   创建时间 1545031151.25 
进程名 crypto                内存利用率 0.0                进程状态 idle       创建时间 1545031151.25 
进程名 kintegrityd           内存利用率 0.0                进程状态 idle       创建时间 1545031151.25 
进程名 kblockd               内存利用率 0.0                进程状态 idle       创建时间 1545031151.25 
进程名 ata_sff               内存利用率 0.0                进程状态 idle       创建时间 1545031151.62 
进程名 md                    内存利用率 0.0                进程状态 idle       创建时间 1545031151.62 
进程名 edac-poller           内存利用率 0.0                进程状态 idle       创建时间 1545031151.62 
进程名 devfreq_wq            内存利用率 0.0                进程状态 idle       创建时间 1545031151.62 
进程名 watchdogd             内存利用率 0.0                进程状态 idle       创建时间 1545031151.62 
进程名 kworker/10:1          内存利用率 0.0                进程状态 idle       创建时间 1545031151.82 
进程名 kworker/15:1          内存利用率 0.0        

进程名 kworker/7:1H          内存利用率 0.0                进程状态 idle       创建时间 1545031154.62 
进程名 kworker/23:1H         内存利用率 0.0                进程状态 idle       创建时间 1545031154.66 
进程名 kworker/22:1H         内存利用率 0.0                进程状态 idle       创建时间 1545031154.66 
进程名 sshd                  内存利用率 0.01969377697326719 进程状态 sleeping   创建时间 1545031154.81 
进程名 gdm3                  内存利用率 0.024303994065835603 进程状态 sleeping   创建时间 1545031154.82 
进程名 upowerd               内存利用率 0.025143321188229324 进程状态 sleeping   创建时间 1545031155.17 
进程名 gdm-session-wor       内存利用率 0.025520410185246794 进程状态 sleeping   创建时间 1545031155.62 
进程名 systemd               内存利用率 0.024121531647923924 进程状态 sleeping   创建时间 1545031155.66 
进程名 (sd-pam)              内存利用率 0.007736406519455177 进程状态 sleeping   创建时间 1545031155.66 
进程名 gdm-x-session         内存利用率 0.016750049964292104 进程状态 sleeping   创建时间 1545031155.69 
进程名 Xorg                  内存利用率 0.14178546287856847 进程状态 sleeping   创建时间 1545031155.7 
进程名 irq/77-nvidia         内存利用

进程名 kworker/29:2          内存利用率 0.0                进程状态 idle       创建时间 1545084646.84 
进程名 systemd               内存利用率 0.024851381319570636 进程状态 sleeping   创建时间 1545355074.55 
进程名 (sd-pam)              内存利用率 0.008782524382148802 进程状态 sleeping   创建时间 1545355074.55 
进程名 wpa_supplicant        内存利用率 0.01734609386280359 进程状态 sleeping   创建时间 1545085031.92 
进程名 kworker/27:3          内存利用率 0.0                进程状态 idle       创建时间 1545362748.92 
进程名 kworker/16:1          内存利用率 0.0                进程状态 idle       创建时间 1546510866.2 
进程名 kworker/30:4          内存利用率 0.0                进程状态 idle       创建时间 1545362751.49 
进程名 tensorboard           内存利用率 1.3253826753880458 进程状态 sleeping   创建时间 1545896400.68 
进程名 kworker/25:2          内存利用率 0.0                进程状态 idle       创建时间 1546038295.07 
进程名 systemd-udevd         内存利用率 0.012942667510535076 进程状态 sleeping   创建时间 1545085060.64 
进程名 packagekitd           内存利用率 0.07387295093184165 进程状态 sleeping   创建时间 1545119047.38 
进程名 jupyter-noteboo       内存利用率 0.22

进程名 ibus-engine-simple    内存利用率 0.0188909423344558 进程状态 sleeping   创建时间 1545716268.9 
进程名 evolution-addressbook-factory  内存利用率 0.07300929548705971 进程状态 sleeping   创建时间 1545716269.23 
进程名 evolution-addressbook-factory-subprocess  内存利用率 0.08034428468710918 进程状态 sleeping   创建时间 1545716269.29 
进程名 gnome-software        内存利用率 0.659443506655195  进程状态 sleeping   创建时间 1545716328.99 
进程名 update-notifier       内存利用率 0.08051458294382675 进程状态 sleeping   创建时间 1545716328.99 
进程名 livepatch-notif       内存利用率 0.0                进程状态 zombie     创建时间 1545716329.08 
进程名 fwupd                 内存利用率 0.06359423472281708 进程状态 sleeping   创建时间 1545716330.58 
进程名 gpg-agent             内存利用率 0.0018367883403108988 进程状态 sleeping   创建时间 1545716343.83 
进程名 deja-dup-monitor      内存利用率 0.09473448737974358 进程状态 sleeping   创建时间 1545716388.95 
进程名 kworker/7:2           内存利用率 0.0                进程状态 idle       创建时间 1545185579.75 
进程名 kworker/15:0          内存利用率 0.0                进程状态 idle       创建时间 1546432795.07 
进程名 kwo